In [ ]:
import pandas as pd
import json

In [ ]:
df = pd.read_csv("../crawl/out/output.csv")
len(df)

## Prepare radio data

Filter to countries with at least 3 stations

In [ ]:
s = df.groupby("country").size() >= 3
s = s[s]

In [ ]:
df = df[df["country"].isin(s.index)]
len(df)

Sample 3 stations for each country

In [ ]:
radio = df.groupby("country").sample(3)

In [ ]:
radio.shape

In [ ]:
len(radio["country"].unique())

In [ ]:
radio.shape

## Matching up

In [ ]:
import geopandas as gpd

In [ ]:
datamaps = pd.read_json("../data/datamaps.json")
ne = gpd.read_file("../data/ne_110m_admin_0_countries.geojson")
centers = gpd.read_file("../data/centers.geojson")

In [ ]:
radio_countries = radio["country"].unique()
len(radio_countries)

### Merge with NE

In [ ]:
df = pd.merge(radio, ne, left_on="country", right_on="SUBUNIT", suffixes=('_radio', '_ne'))
df.shape

In [ ]:
radio[~radio["country"].isin(df["country"])]["country"].unique()

### Filter to datamaps

In [ ]:
df2 = df[df["ADM0_A3"].isin(datamaps["id"])]
df2.shape

In [ ]:
df[~df["country"].isin(df2["country"])]["country"].unique()

### Filter to centers

In [ ]:
df3 = df2[df2["ISO_A2_EH"].isin(centers["ISO"])]
df3.shape

In [ ]:
df2[~df2["country"].isin(df3["country"])]["country"].unique()

## Finish

In [ ]:
out = df3

In [ ]:
use_cols = list(radio.columns) + ['ADM0_A3', 'ISO_A2_EH']

In [ ]:
out = out[use_cols]

In [ ]:
out = out.rename({
    "ADM0_A3": "three_code",
    "ISO_A2_EH": "two_code",
}, axis=1)

In [ ]:
out